In [10]:
!pip install vosk

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install gtts pygame

Defaulting to user installation because normal site-packages is not writeable


In [32]:
!pip install fpdf

Defaulting to user installation because normal site-packages is not writeable


<h1>TRANSCRIPCIÓN DE VOZ A TEXTO / PARA LLENAR UNA PLANTILLA EN PDF.</h1>

Que se usa:
+ vosk
+ gradio (interfaz)
+ voz (pygame)

**<h2>Universidad Nacional de Loja</h2>**

----

 **<h3>Materia:** Human Perception in Computer Vision</h3>
  **<h3>Integrantes:**</h3>
+ Luis Bermeo
+ Anthony Luzuriaga
+ Gerar Quizhpe
+ Alex Viñamagua

# FUNCIONES GENERAL (convertir a num, pdf, word)

In [23]:
def convertir_a_numero(texto):
    # Diccionarios para mapeo de palabras a números
    unidades = {
        'uno': 1, 'un': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
        'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
        'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
        'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19, 'veinte': 20,
        'veintiuno': 21, 'veintidós': 22, 'veintitrés': 23, 'veinticuatro': 24, 'veinticinco': 25,
        'veintiséis': 26, 'veintisiete': 27, 'veintiocho': 28, 'veintinueve': 29
    }
    decenas = {
        'treinta': 30, 'cuarenta': 40, 'cincuenta': 50, 'sesenta': 60,
        'setenta': 70, 'ochenta': 80, 'noventa': 90
    }
    centenas = {
        'ciento': 100, 'doscientos': 200, 'trescientos': 300, 'cuatrocientos': 400, 'quinientos': 500,
        'seiscientos': 600, 'setecientos': 700, 'ochocientos': 800, 'novecientos': 900
    }
    especiales = {
        'cien': 100, 'mil': 1000, 'millón': 1000000, 'millones': 1000000, 'cero': 0
    }

    palabras = texto.split()
    total = 0
    temporal = 0
    resultado = ""
    decimal_part = False

    for palabra in palabras:
        if palabra in unidades:
            if decimal_part:
                resultado += str(unidades[palabra])
            else:
                temporal += unidades[palabra]
        elif palabra in decenas:
            temporal += decenas[palabra]
        elif palabra in centenas:
            temporal += centenas[palabra]
        elif palabra in especiales:
            if palabra == 'mil':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000
                temporal = 0
            elif palabra == 'millón' or palabra == 'millones':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000000
                temporal = 0
            else:
                temporal += especiales[palabra]
        elif palabra == 'punto' or palabra == 'coma':  # Manejar separadores decimales
            if not decimal_part:
                resultado += str(total + temporal) + "."
                decimal_part = True
                temporal = 0
        elif palabra == 'y':
            continue
        else:
            return "0"

    if not decimal_part:
        resultado += str(total + temporal)
    return resultado

In [21]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'FICHA DE DESCRIPCIÓN DE AFLORAMIENTOS', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def table(self, data):
        self.set_font('Arial', 'B', 12)
        # Header, el total es de 190 el largo del documento. osea 19cm.
        self.cell(40, 10, 'Responsable:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['responsable'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(40, 10, 'Afloramiento N°:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['afloramiento'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(40, 10, 'Carta Topográfica:', 1)
        self.set_font('Arial', '', 12)
        self.cell(150, 10, data['carta_topografica'], 1)
        self.ln()
        
        ######################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'UBICACIÓN', 1, 1, 'C')

        self.cell(30, 10, 'Provincia:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['provincia'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Cantón:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['canton'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Parroquia:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['parroquia'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Sector:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['sector'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'DATUM:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, "WGS 1984", 1)
        self.ln()
        #######################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Localización', 1, 1, 'C')

        self.cell(30, 10, 'X:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['localizacion_x'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Y:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, data['localizacion_y'], 1)
        self.ln()

        self.set_font('Arial', 'B', 12)
        self.cell(30, 10, 'Z:', 1)
        self.set_font('Arial', '', 12)
        self.cell(160, 10, str(data['localizacion_z'])+" m", 1)
        self.ln()
        ########################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Tipo de afloramiento', 1, 1, 'C')

        self.cell(45, 10, 'Alforamiento Natural:', 1)
        self.set_font('Arial', '', 12)
        self.cell(45, 10, data['tipo_afloramiento_natural'], 1)

        self.set_font('Arial', 'B', 12)
        self.cell(60, 10, 'Alforamiento Antropogenico:', 1)
        self.set_font('Arial', '', 12)
        self.cell(40, 10, data['tipo_afloramiento_antropogenico'], 1)
        self.ln()
        ########################################################################
        self.set_font('Arial', 'B', 12)
        self.cell(190, 10, 'Dimensiones', 1, 1, 'C')

        self.cell(20, 10, 'Largo:', 1)
        self.set_font('Arial', '', 12)
        self.cell(75, 10, str(data['largo'])+" m",1)

        self.set_font('Arial', 'B', 12)
        self.cell(20, 10, 'Ancho:', 1)
        self.set_font('Arial', '', 12)
        self.cell(75, 10, str(data['ancho'])+" m", 1)
        self.ln()
        
# Función para crear y guardar el PDF
def create_pdf(data):
    pdf = PDF()
    pdf.add_page()
    pdf.table(data)
    output_path = "plantilla_ejemplo_datos.pdf"
    pdf.output(output_path)
    return output_path

In [3]:
#Generar Word

from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def create_word(data):
    doc = Document()

    # Header
    header = doc.add_heading(level=1)
    header_run = header.add_run('FICHA DE DESCRIPCIÓN DE AFLORAMIENTOS')
    header_run.font.bold = True
    header_run.font.size = Pt(12)
    header.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Function to add table rows
    def add_table_row(table, cells):
        row_cells = table.add_row().cells
        for idx, cell_text in enumerate(cells):
            row_cells[idx].text = cell_text
    
    # Function para cambiar el color del texto 
    def add_bold_heading(doc, text):
        paragraph = doc.add_heading(level=2)
        run = paragraph.add_run(text)
        run.font.bold = True
        run.font.color.rgb = RGBColor(0, 0, 0)  # Negro

    # Table 1: General Information
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['Responsable:', data['responsable']])
    add_table_row(table, ['Afloramiento N°:', data['afloramiento']])
    add_table_row(table, ['Carta Topográfica:', data['carta_topografica']])

    # Table 2: Ubicación
    doc.add_heading('UBICACIÓN', level=2)
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['Provincia:', data['provincia']])
    add_table_row(table, ['Cantón:', data['canton']])
    add_table_row(table, ['Parroquia:', data['parroquia']])
    add_table_row(table, ['Sector:', data['sector']])
    #add_table_row(table, ['DATUM:', data['datum']])

    # Table 3: Localización
    doc.add_heading('Localización', level=2)
    table = doc.add_table(rows=0, cols=2)
    table.style = 'Table Grid'
    add_table_row(table, ['X:', data['localizacion_x']])
    add_table_row(table, ['Y:', data['localizacion_y']])
    add_table_row(table, ['Z:', data['localizacion_z']])

    # Table 4: Tipo de afloramiento
    doc.add_heading('Tipo de afloramiento', level=2)
    table = doc.add_table(rows=0, cols=4)
    table.style = 'Table Grid'
    add_table_row(table, ['Alforamiento Natural:', data['tipo_afloramiento_natural'], 'Alforamiento Antropogenico:', data['tipo_afloramiento_antropogenico']])

    # Table 5: Dimensiones
    doc.add_heading('Dimensiones', level=2)
    table = doc.add_table(rows=0, cols=4)
    table.style = 'Table Grid'
    add_table_row(table, ['Largo:', data['largo'], 'Ancho:', data['ancho']])

    # Save the document
    output_path = 'plantilla_ejemplo_datos.docx'
    doc.save(output_path)
    return output_path

# CODIGO DE LA VOZ  Y EL MODELO


In [24]:
import os
import wave
import json
import vosk
import pyaudio
import time
import pygame
from gtts import gTTS
import gradio as gr

In [6]:
# Inicializar Pygame
pygame.mixer.init()
is_recording = False

In [7]:
def play_sound(file):
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

In [8]:
def text_to_audio(text, filename):
    tts = gTTS(text=text, lang='es')
    tts.save(filename)

In [22]:
import tempfile
#Este código toma un texto, lo convierte en un archivo de audio temporal, y luego reproduce ese archivo utilizando Pygame
def play_text(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        temp_filename = temp_audio.name
        text_to_audio(text, temp_filename)

    # Asegurarse de que el archivo se ha guardado antes de cargarlo
    if os.path.exists(temp_filename):
        pygame.mixer.music.load(temp_filename)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        time.sleep(0.5)  # Pequeño retraso para asegurarse de que Pygame ha terminado de usar el archivo
    else:
        print(f"Error: No se pudo encontrar el archivo {temp_filename}")

In [11]:
# Función para capturar voz, modelos en : https://alphacephei.com/vosk/models
def capture_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=4096)
    stream.start_stream()  
    CHUNK = 1024  # Reduce el tamaño del chunk
    #model = vosk.Model("model/vosk-model-es-0.42/")
    rec = vosk.KaldiRecognizer(model, 16000)
    
    print("Di tu comando:")   
    result_text = ""
    while True:
        # Lee datos del flujo de audio
        data = stream.read(CHUNK, exception_on_overflow=False)   #data = stream.read(4096)
        if len(data) == 0:
            continue
        if rec.AcceptWaveform(data):
            result = rec.Result()
            text = json.loads(result)["text"]
            result_text += text + " "
            break
    stream.stop_stream()
    stream.close()
    p.terminate()
    return result_text.strip()

In [12]:
# Función para asignar valores a variables
def assign_variable(variable_name):
    prompt = f"Por favor, di el valor para {variable_name}:"
    play_text(prompt)
    value = capture_audio()
    captured_message = f"Capturado: {value}"
    play_text(captured_message)
    if variable_name in ['afloramiento', 'localizacion_x', 'localizacion_y', 'localizacion_z', 'largo', 'ancho']:
        value = convertir_a_numero(value)   
    # Manejar el caso especial para el tipo de afloramiento
    if variable_name in ['tipo_afloramiento_natural', 'tipo_afloramiento_antropogenico']:
        if value.lower() == 'si':
            value = 'X'
        else:
            value = ''
    data[variable_name] = value

In [13]:
def update_data(field, value):
    data[field] = value

In [14]:
# Diccionario para guardar variables
model = vosk.Model("model/vosk-model-es-0.42/")

data = {
    'responsable': '',
    'afloramiento': '',
    'carta_topografica': '',
    'provincia': '',
    'canton': '',
    'parroquia': '',
    'sector': '',
    'localizacion_x': '',
    'localizacion_y': '',
    'localizacion_z': '',
    'tipo_afloramiento_natural': '',
    'tipo_afloramiento_antropogenico': '',
    'largo': '',
    'ancho': '',
}

In [15]:
def main():
    for key in data.keys():
        assign_variable(key)

In [16]:
# Generar Archivos
def generar_archivoPDF():
    output_path_pdf = create_pdf(data)
    return output_path_pdf


In [17]:
# Generar Archivos
def generar_archivoWord():
    output_path_word = create_word(data)
    return output_path_word

In [20]:
with gr.Blocks() as demo:
    
    gr.HTML("""
        <style>
        .gradio-container {
            font-family: Arial, sans-serif;
            background-color: #f5f5f5;
            padding: 20px;
        }
        h1 { gr.HTML("<img src='logoRocosos.jpg' alt='Logo' width='200'>")
            color: #333;
        }
        .gr-button {
            background-color: #4CAF50;
            color: white;
        }
        </style>
    """)
    
    #gr.HTML("<img src='logoRocosos.jpg' alt='Logo' width='200'>")
    
    # Añade un encabezado Markdown a la aplicación
    gr.Markdown("# Aplicación de Transcripción de Voz a Texto para generar una Plantilla PDF")
    
    # Crea una pestaña llamada "Formulario"
    with gr.Tab("Formulario"):
        # Crea una fila para organizar los elementos
        with gr.Row():
            # Primera columna de la fila
            with gr.Column():
                # Crea un botón de inicio de grabación para cada clave en el diccionario 'data'
                start_btns = [gr.Button(f"Iniciar grabación {key}") for key in data.keys()]
                # Crea un botón de pausa
                #pause_btn = gr.Button("Pausar")
                # Crea un botón de detener
                #stop_btn = gr.Button("Detener")
            
            # Segunda columna de la fila
            with gr.Column():
                # Crea campos de texto (textboxes) para cada dato en el formulario
                responsable = gr.Textbox(label="Responsable")
                afloramiento = gr.Textbox(label="Afloramiento N°")
                carta_topografica = gr.Textbox(label="Carta Topográfica")
                provincia = gr.Textbox(label="Provincia")
                canton = gr.Textbox(label="Cantón")
                parroquia = gr.Textbox(label="Parroquia")
                sector = gr.Textbox(label="Sector")
                localizacion_x = gr.Textbox(label="Localización X")
                localizacion_y = gr.Textbox(label="Localización Y")
                localizacion_z = gr.Textbox(label="Localización Z")
                tipo_afloramiento_natural = gr.Textbox(label="Tipo de afloramiento natural")
                tipo_afloramiento_antropogenico = gr.Textbox(label="Tipo de afloramiento antropogénico")
                largo = gr.Textbox(label="Largo")
                ancho = gr.Textbox(label="Ancho")
                # Crea un botón para generar PDF y Word
                generate_pdf_btn = gr.Button("Generar PDF")
                generate_word_btn = gr.Button("Generar Word")

                # Mapea los textboxes a las claves del diccionario 'data'
                textboxes = {
                    'responsable': responsable,
                    'afloramiento': afloramiento,
                    'carta_topografica': carta_topografica,
                    'provincia': provincia,
                    'canton': canton,
                    'parroquia': parroquia,
                    'sector': sector,
                    'localizacion_x': localizacion_x,
                    'localizacion_y': localizacion_y,
                    'localizacion_z': localizacion_z,
                    'tipo_afloramiento_natural': tipo_afloramiento_natural,
                    'tipo_afloramiento_antropogenico': tipo_afloramiento_antropogenico,
                    'largo': largo,
                    'ancho': ancho,
                }

        # Asocia los botones de inicio de grabación con la función 'assign_variable' para cada campo
        for key, btn in zip(data.keys(), start_btns):
            btn.click(lambda k=key: assign_variable(k), inputs=None, outputs=textboxes[key])

        # Asocia el botón de generar PDF con la función 'generar_archivos'
        generate_pdf_btn.click(generar_archivoPDF)
        generate_word_btn.click(generar_archivoWord)
    
        
    # Añade una sección de instrucciones Markdown
    gr.Markdown("## Instrucciones")
    gr.Markdown("""
    1. Haz clic en "Iniciar grabación" junto al campo que quieres llenar.
    2. Di el valor correspondiente.
    3. Verifica que el campo se haya llenado correctamente.
    4. Repite para todos los campos necesarios.
    5. Haz clic en "Generar PDF y Word" cuando hayas completado todos los campos.
    """)

In [39]:
if __name__ == "__main__":
    demo.launch(server_port=7871)
    #demo.close()
    #main()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Di tu comando:
Di tu comando:
Di tu comando:
Di tu comando:
Di tu comando:
Di tu comando:
